In [ ]:
# IMDb movie poster base url
IMDB_BASE_URL = "https://image.tmdb.org/t/p/original"

In [2]:
!pip install IMDbPY

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 2.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
import requests
from imdb import IMDb

In [4]:
movie_df = pd.read_csv("/content/drive/Shareddrives/5500 Database Project/datasets/movies/movies_metadata.csv", nrows=100)

In [8]:
with pd.option_context("display.max_columns", None):
  print(movie_df.head())

   adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497         

# Part 1: Data Loading

In [ ]:
# loads top n rows of data
song_df = pd.read_csv("/content/drive/Shareddrives/5500 Database Project/datasets/songs.csv", nrows=1000)
tv_df = pd.read_csv("/content/drive/Shareddrives/5500 Database Project/datasets/tv_series.csv", nrows=100)

In [ ]:
song_df.head()

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",1,en,en,en
1,Can I Live,rap,JAY-Z,1996,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",3,en,en,en
2,Forgive Me Father,rap,Fabolous,2003,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,4,en,en,en
3,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,5,en,en,en
4,Fly In,rap,Lil Wayne,2005,78271,{},"[Intro]\nSo they ask me\n""Young boy\nWhat you ...",6,en,en,en


In [ ]:
# screens english songs
song_en_df = song_df[song_df["language"]=="en"][["title", "tag", "artist"]]

In [ ]:
tv_df.head()

,Series Title,Release Year,Runtime,Genre,Rating,Cast,Synopsis
0,Wednesday,(2022– ),45 min,"Comedy, Crime, Fantasy",8.2,"Jenna Ortega, Hunter Doohan, Percy Hynes White...","Follows Wednesday Addams' years as a student, ..."
1,Yellowstone,(2018– ),60 min,"Drama, Western",8.7,"Kevin Costner, Luke Grimes, Kelly Reilly, Wes ...",A ranching family in Montana faces off against...
2,The White Lotus,(2021–2023),60 min,"Comedy, Drama",7.9,"Jennifer Coolidge, Jon Gries, F. Murray Abraha...","Set in a tropical resort, it follows the explo..."
3,1923,(2022–2023),60 min,"Drama, Western",8.6,"Harrison Ford, Helen Mirren, Brandon Sklenar, ...",The Duttons face a new set of challenges in th...
4,Jack Ryan,(2018– ),60 min,"Action, Drama, Thriller",8.0,"John Krasinski, Wendell Pierce, Michael Kelly,...","Up-and-coming CIA analyst, Jack Ryan, is thrus..."


# Part 2: Artist Image (Spotify Open API)

## 2.1. Connecting to Spotify

In [ ]:
# sets personal info from spotify developer
CLIENT_ID = "19e425a7eac64c6692fd4e5076e196b8"
CLIENT_SECRET = "b954d8722b394093bc55a6e2864add2e"

# authenticates my app
auth_url = "https://accounts.spotify.com/api/token"

auth_response = requests.post(auth_url, {
    "grant_type": "client_credentials",
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
})

# converts the response to JSON
auth_response_data = auth_response.json()

# saves the access token
access_token = auth_response_data["access_token"]

## 2.2. Extracting Artist Image URL

In [ ]:
# finds artist's Spotify ID
artist_name = "Taylor Swift"
search_url = "https://api.spotify.com/v1/search"

headers = {"Authorization": f"Bearer {access_token}"}
params = {
    "q": artist_name,
    "type": "artist",
    "limit": 1
}

response = requests.get(search_url, headers=headers, params=params)
search_results = response.json()

# extracts artist ID
artist_id = search_results["artists"]["items"][0]["id"] if search_results["artists"]["items"] else None
print(artist_id)

06HL4z0CvFAxyc27GXpf02


In [ ]:
# makes a request for the artist's data
artist_url = f"https://api.spotify.com/v1/artists/{artist_id}"

response = requests.get(artist_url, headers=headers)
artist_data = response.json()

In [ ]:
# extracts image URL
image_url = artist_data["images"][0]["url"] if artist_data["images"] else None
print(image_url)

https://i.scdn.co/image/ab6761610000e5eb859e4c14fa59296c8649e0e4


## 2.3. Extracting All Artists' Images

In [ ]:
artist_names = list(song_en_df["artist"])
artist_ids = []

# finds artists' Spotify ID
for artist in artist_names:
  params = {
    "q": artist,
    "type": "artist",
    "limit": 1
  }
  response = requests.get(search_url, headers=headers, params=params)
  search_results = response.json()

  # extracts artist ID and adds it to the list
  artist_id = search_results["artists"]["items"][0]["id"] if search_results["artists"]["items"] else None
  artist_ids.append(artist_id)

In [ ]:
print(artist_ids[:3])

['7iMvwE8qANp3aIfAGKEAwS', '3nFkdlSjzX9mRTtwJOzDYB', '0YWxKQj2Go9CGHCp77UOyy']


In [ ]:
# extracts image URLs
image_urls=[]

for idx, id in enumerate(artist_ids):
  artist_url = f"https://api.spotify.com/v1/artists/{id}"

  response = requests.get(artist_url, headers=headers)
  artist_data = response.json()

  try:
    image_url = artist_data["images"][0]["url"] if artist_data["images"] else None
    image_urls.append(image_url)

  except KeyError:
    # if there is an invalid ID, adds "no image" to the list and returns to the loop
    # we can handle this on the client side by converting it to the default image
    print(f"Index {idx}: Invalid Artist ID")
    image_urls.append("no image")
    continue

Index 140: Invalid Artist ID


In [ ]:
# creates a new column with image url data
song_en_df["image"] = image_urls

In [ ]:
song_en_df.head()

,title,tag,artist,image
0,Killa Cam,rap,Cam'ron,https://i.scdn.co/image/ab6761610000e5eb7455f2...
1,Can I Live,rap,JAY-Z,https://i.scdn.co/image/ab6761610000e5ebc75afc...
2,Forgive Me Father,rap,Fabolous,https://i.scdn.co/image/ab6761610000e5ebece560...
3,Down and Out,rap,Cam'ron,https://i.scdn.co/image/ab6761610000e5eb7455f2...
4,Fly In,rap,Lil Wayne,https://i.scdn.co/image/ab6761610000e5ebc1c08e...


In [ ]:
song_en_df[song_en_df["image"] == "no image"]

,title,tag,artist,image
141,Forever,rap,"Drake, Kanye West, Lil Wayne & Eminem",no image


# Part 3: TV Series Poster (IMDbPY Library)

In [ ]:
# creates an instance of the IMDb class
imdb = IMDb()

# searches for the TV series by title
series_name = "Wednesday"
search_result = imdb.search_movie(series_name)[0].movieID

# gets series data
series_data = imdb.get_movie(search_result)

# gets poster url from the data
poster_url = series_data["full-size cover url"]
print(poster_url)

https://m.media-amazon.com/images/M/MV5BM2ZmMjEyZmYtOGM4YS00YTNhLWE3ZDMtNzQxM2RhNjBlODIyXkEyXkFqcGdeQXVyMTUzMTg2ODkz.jpg


In [ ]:
series_names = list(tv_df["Series Title"])
poster_urls = []

for idx, name in enumerate(series_names):
  search_result = imdb.search_movie(name)[0].movieID
  search_data = imdb.get_movie(search_result)

  # checks if the poster URL is available
  if "full-size cover url" in series_data.keys():
    poster_url = series_data["full-size cover url"]
    poster_urls.append(poster_url)
  else:
    # if there is no poster, adds "no poster" to the list
    # we can handle this on the client side by converting it to the default image
    print(f"Indext {idx}: Poster URL not found")
    poster_urls.append("no poster")

In [ ]:
print(poster_urls[:3])

['https://m.media-amazon.com/images/M/MV5BM2ZmMjEyZmYtOGM4YS00YTNhLWE3ZDMtNzQxM2RhNjBlODIyXkEyXkFqcGdeQXVyMTUzMTg2ODkz.jpg', 'https://m.media-amazon.com/images/M/MV5BM2ZmMjEyZmYtOGM4YS00YTNhLWE3ZDMtNzQxM2RhNjBlODIyXkEyXkFqcGdeQXVyMTUzMTg2ODkz.jpg', 'https://m.media-amazon.com/images/M/MV5BM2ZmMjEyZmYtOGM4YS00YTNhLWE3ZDMtNzQxM2RhNjBlODIyXkEyXkFqcGdeQXVyMTUzMTg2ODkz.jpg']


In [ ]:
# creates a new column
tv_df["poster"] = poster_urls

In [ ]:
tv_df.head()

,Series Title,Release Year,Runtime,Genre,Rating,Cast,Synopsis,poster
0,Wednesday,(2022– ),45 min,"Comedy, Crime, Fantasy",8.2,"Jenna Ortega, Hunter Doohan, Percy Hynes White...","Follows Wednesday Addams' years as a student, ...",https://m.media-amazon.com/images/M/MV5BM2ZmMj...
1,Yellowstone,(2018– ),60 min,"Drama, Western",8.7,"Kevin Costner, Luke Grimes, Kelly Reilly, Wes ...",A ranching family in Montana faces off against...,https://m.media-amazon.com/images/M/MV5BM2ZmMj...
2,The White Lotus,(2021–2023),60 min,"Comedy, Drama",7.9,"Jennifer Coolidge, Jon Gries, F. Murray Abraha...","Set in a tropical resort, it follows the explo...",https://m.media-amazon.com/images/M/MV5BM2ZmMj...
3,1923,(2022–2023),60 min,"Drama, Western",8.6,"Harrison Ford, Helen Mirren, Brandon Sklenar, ...",The Duttons face a new set of challenges in th...,https://m.media-amazon.com/images/M/MV5BM2ZmMj...
4,Jack Ryan,(2018– ),60 min,"Action, Drama, Thriller",8.0,"John Krasinski, Wendell Pierce, Michael Kelly,...","Up-and-coming CIA analyst, Jack Ryan, is thrus...",https://m.media-amazon.com/images/M/MV5BM2ZmMj...


🫠 The problem is it takes forever... (especially with the TV series)